# Hypothesis Testing
This code does the following:
* Reads the FC files for all the subjects
* Z-Standardize all the voxel-roi correlation values of each ROI
* Perform two tailed t-test for each voxel-roi pair correlation across subjects (Autism vs TD)

In [3]:
import os
from os.path import join as opj
# from nipype.interfaces import afni
import nibabel as nib
import json    
import numpy as np
import os

In [21]:
# Paths

path_cwd = os.getcwd()
path_split_list = path_cwd.split('/')
s = path_split_list[0:-1] # for getting to the parent dir of pwd
s_main = path_split_list[0:-2]
s = opj('/',*s) # *s converts list to path, # very important to add '/' in the begining so it is read as directory later
s_main = opj('/',*s_main)


In [22]:
s_main

'/home1/varunk'

In [6]:

# json_path = opj(data_directory,'task-rest_bold.json')

json_path = '../scripts/json/paths.json'
with open(json_path, 'rt') as fp:
    task_info = json.load(fp)
    

In [7]:


base_directory = opj(s,task_info["base_directory_for_results"]) 
motion_correction_bet_directory = task_info["motion_correction_bet_directory"]
parent_wf_directory = task_info["parent_wf_directory"]
# functional_connectivity_directory = task_info["functional_connectivity_directory"]
functional_connectivity_directory = 'temp_fc'
coreg_reg_directory = task_info["coreg_reg_directory"]
atlas_resize_reg_directory = task_info["atlas_resize_reg_directory"]
data_directory = opj(s,task_info["data_directory"])
datasink_name = task_info["datasink_name"]
# fc_datasink_name = task_info["fc_datasink_name"]
fc_datasink_name = 'temp_dataSink'
atlasPath = opj(s,task_info["atlas_path"])

hypothesis_test_dir = opj(base_directory, task_info["hypothesis_test_dir"])



### Use python pandas to read the composite phenotype file 

In [8]:
#  Runall:
import pandas as pd

df = pd.read_csv('/home1/varunk/data/ABIDE1/RawDataBIDs/composite_phenotypic_file.csv') # , index_col='SUB_ID'

df = df.sort_values(['SUB_ID'])
# df = df.sort_values(['SUB+AF8-ID'])

# These subjects didn't have anatomical highres file: 
# 51232','51233','51242','51243','51244','51245','51246','51247','51270','51310'

bugs = ['51232','51233','51242','51243','51244','51245','51246','51247','51270','51310', '50045']
# sub 50045 has fewer ROIs coz of more shift of head

In [9]:
# '0051242' in bugs

In [42]:
# df

In [45]:
tr_paths = opj(s_main,task_info["base_directory_for_results"],datasink_name,'tr_paths','tr_list.npy')
tr_list = np.load(tr_paths)
# tr_list.max()
# tr_list
# len(np.where(tr_list == 1.5)[0]),len(np.where(tr_list == 3.0)[0])
# np.where(tr_list == 3.0)

df_array = df.as_matrix(['SUB_ID'])

df_array[np.where(tr_list == 3.0)]

array([[51201],
       [51202],
       [51203],
       [51204],
       [51205],
       [51206],
       [51207],
       [51208],
       [51209],
       [51210],
       [51211],
       [51212],
       [51213],
       [51214],
       [51215],
       [51216],
       [51217],
       [51218],
       [51219],
       [51220],
       [51221],
       [51222],
       [51223],
       [51224],
       [51225],
       [51226],
       [51227],
       [51228],
       [51229],
       [51230],
       [51231],
       [51232],
       [51233],
       [51234],
       [51235],
       [51236],
       [51237],
       [51238],
       [51239],
       [51240],
       [51241],
       [51242],
       [51243],
       [51244],
       [51245],
       [51246],
       [51247],
       [51248],
       [51249],
       [51250],
       [51251],
       [51252],
       [51253],
       [51254],
       [51255],
       [51256],
       [51257],
       [51258],
       [51259],
       [51260],
       [51261],
       [51262],
       [

In [ ]:
# tr = 
/home1/varunk/results_again_again/ABIDE1_Preprocess_Datasink/tr_paths/tr_list.npy

In [25]:
# selecting Autistic males(DSM IV) of age <= 18 years 
df_aut_lt18_m = df.loc[(df['SEX'] == 1) & (df['AGE_AT_SCAN'] <=18) & (df['DSM_IV_TR'] == 1)]

In [29]:
df_aut_lt18_m.shape

(214, 74)

In [15]:
df_aut_lt18_m_eyesopen = df.loc[(df['SEX'] == 1) & (df['AGE_AT_SCAN'] <=18) & (df['DSM_IV_TR'] == 1) & df['EYE_STATUS_AT_SCAN'] == 1]
df_aut_lt18_m_eyesopen

,SITE_ID,SUB_ID,DX_GROUP,DSM_IV_TR,AGE_AT_SCAN,SEX,HANDEDNESS_CATEGORY,HANDEDNESS_SCORES,FIQ,VIQ,...,WISC_IV_BLK_DSN_SCALED,WISC_IV_PIC_CON_SCALED,WISC_IV_MATRIX_SCALED,WISC_IV_DIGIT_SPAN_SCALED,WISC_IV_LET_NUM_SCALED,WISC_IV_CODING_SCALED,WISC_IV_SYM_SCALED,EYE_STATUS_AT_SCAN,AGE_AT_MPRAGE,BMI
577,SDSU,50183,1,1,14.14,1,R,NaN,139.0,128.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
586,SDSU,50192,1,1,12.99,1,R,NaN,85.0,93.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
810,UM_1,50272,1,1,14.20,1,R,NaN,98.5,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
811,UM_1,50273,1,1,16.80,1,R,NaN,112.5,109.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
812,UM_1,50274,1,1,14.20,1,R,NaN,111.5,106.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
813,UM_1,50275,1,1,11.50,1,L,NaN,85.0,106.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
815,UM_1,50277,1,1,15.40,1,L,NaN,107.5,96.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
817,UM_1,50279,1,1,12.20,1,R,NaN,87.5,75.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
818,UM_1,50280,1,1,15.60,1,-9999,NaN,85.0,81.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
820,UM_1,50282,1,1,15.20,1,NaN,NaN,121.0,118.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN


In [136]:
df_aut_lt18_m_eyesclosed = df.loc[(df['SEX'] == 1) & (df['AGE_AT_SCAN'] <=18) & (df['DSM_IV_TR'] == 1) & (df['EYE_STATUS_AT_SCAN'] == 2)]
df_aut_lt18_m_eyesclosed.shape

(40, 74)

In [ ]:
bugs = ['50045'] # ROI Missing in population under study
# ROI Missing:
# 253	./_subject_id_0050655/func2std_xform/0050655_fc_map_flirt.nii.gz
# 254	./_subject_id_0050645/func2std_xform/0050645_fc_map_flirt.nii.gz
# 256	./_subject_id_0050651/func2std_xform/0050651_fc_map_flirt.nii.gz
# 256	./_subject_id_0050658/func2std_xform/0050658_fc_map_flirt.nii.gz
# 259	./_subject_id_0050643/func2std_xform/0050643_fc_map_flirt.nii.gz
# 259	./_subject_id_0050652/func2std_xform/0050652_fc_map_flirt.nii.gz
# 270	./_subject_id_0050644/func2std_xform/0050644_fc_map_flirt.nii.gz
# 270	./_subject_id_0051558/func2std_xform/0051558_fc_map_flirt.nii.gz
# 273	./_subject_id_0050648/func2std_xform/0050648_fc_map_flirt.nii.gz
# 273	./_subject_id_0050667/func2std_xform/0050667_fc_map_flirt.nii.gz
# 273	./_subject_id_0050045/func2std_xform/0050045_fc_map_flirt.nii.gz
# 273	./_subject_id_0051575/func2std_xform/0051575_fc_map_flirt.nii.gz
# 273	./_subject_id_0050657/func2std_xform/0050657_fc_map_flirt.nii.gz
# 273	./_subject_id_0050661/func2std_xform/0050661_fc_map_flirt.nii.gz
# 273	./_subject_id_0050653/func2std_xform/0050653_fc_map_flirt.nii.gz
# 273	./_subject_id_0050669/func2std_xform/0050669_fc_map_flirt.nii.gz


In [135]:
df_td_lt18_m_eyesclosed = df.loc[(df['SEX'] == 1) & (df['AGE_AT_SCAN'] <=18) & (df['DSM_IV_TR'] == 0) & (df['EYE_STATUS_AT_SCAN'] == 2)]
df_td_lt18_m_eyesclosed

,SITE_ID,SUB_ID,DX_GROUP,DSM_IV_TR,AGE_AT_SCAN,SEX,HANDEDNESS_CATEGORY,HANDEDNESS_SCORES,FIQ,VIQ,...,WISC_IV_BLK_DSN_SCALED,WISC_IV_PIC_CON_SCALED,WISC_IV_MATRIX_SCALED,WISC_IV_DIGIT_SPAN_SCALED,WISC_IV_LET_NUM_SCALED,WISC_IV_CODING_SCALED,WISC_IV_SYM_SCALED,EYE_STATUS_AT_SCAN,AGE_AT_MPRAGE,BMI
520,PITT,50031,2,0,12.92,1,R,NaN,106.0,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN
522,PITT,50033,2,0,12.15,1,R,NaN,98.0,98.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN
523,PITT,50034,2,0,14.77,1,R,NaN,97.0,98.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN
524,PITT,50035,2,0,17.36,1,R,NaN,107.0,109.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN
532,PITT,50043,2,0,13.78,1,R,NaN,113.0,115.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN
533,PITT,50044,2,0,17.13,1,R,NaN,110.0,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN
534,PITT,50045,2,0,15.70,1,R,NaN,114.0,115.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN
536,PITT,50047,2,0,15.35,1,R,NaN,103.0,95.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN
537,PITT,50048,2,0,11.81,1,R,NaN,107.0,98.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN
539,PITT,50050,2,0,14.37,1,L,NaN,113.0,104.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN


In [131]:
# df.loc[(df['SEX'] == 1) & (df['AGE_AT_SCAN'] <=18) & (df['DSM_IV_TR'] == 1) ]

In [91]:
df_aut_lt18_m_eyesopen.shape

(174, 74)

In [92]:
df_td_lt18_m_eyesopen = df.loc[(df['SEX'] == 1) & (df['AGE_AT_SCAN'] <=18) & (df['DSM_IV_TR'] == 0) & df['EYE_STATUS_AT_SCAN'] == 1]

In [132]:
# df_td_lt18_m_eyesopen['SUB_ID'][100:]

In [94]:
df.loc[(df['EYE_STATUS_AT_SCAN'] == 1)].shape[0] + df.loc[(df['EYE_STATUS_AT_SCAN'] == 2)].shape[0]  

1112

In [95]:
df_aut_lt18_m[103:106]

,SITE_ID,SUB_ID,DX_GROUP,DSM_IV_TR,AGE_AT_SCAN,SEX,HANDEDNESS_CATEGORY,HANDEDNESS_SCORES,FIQ,VIQ,...,WISC_IV_BLK_DSN_SCALED,WISC_IV_PIC_CON_SCALED,WISC_IV_MATRIX_SCALED,WISC_IV_DIGIT_SPAN_SCALED,WISC_IV_LET_NUM_SCALED,WISC_IV_CODING_SCALED,WISC_IV_SYM_SCALED,EYE_STATUS_AT_SCAN,AGE_AT_MPRAGE,BMI
178,LEUVEN_2,50752,1,1,14.7,1,L,NaN,NaN,92.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN
179,LEUVEN_2,50753,1,1,14.3,1,R,NaN,NaN,78.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN
180,LEUVEN_2,50754,1,1,12.3,1,R,NaN,NaN,97.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN


In [96]:
df[520:550]

,SITE_ID,SUB_ID,DX_GROUP,DSM_IV_TR,AGE_AT_SCAN,SEX,HANDEDNESS_CATEGORY,HANDEDNESS_SCORES,FIQ,VIQ,...,WISC_IV_BLK_DSN_SCALED,WISC_IV_PIC_CON_SCALED,WISC_IV_MATRIX_SCALED,WISC_IV_DIGIT_SPAN_SCALED,WISC_IV_LET_NUM_SCALED,WISC_IV_CODING_SCALED,WISC_IV_SYM_SCALED,EYE_STATUS_AT_SCAN,AGE_AT_MPRAGE,BMI
61,CMU,50666,2,0,31.0,1,R,NaN,107.0,101.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN
62,CMU,50667,2,0,40.0,1,R,NaN,128.0,122.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN
63,CMU,50668,2,0,25.0,1,R,NaN,110.0,109.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN
64,CMU,50669,2,0,30.0,2,R,NaN,126.0,123.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN
120,LEUVEN_1,50682,2,0,23.0,1,R,NaN,128.0,118.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
121,LEUVEN_1,50683,2,0,24.0,1,R,NaN,106.0,113.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
122,LEUVEN_1,50685,2,0,23.0,1,R,NaN,112.0,114.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
123,LEUVEN_1,50686,1,1,19.0,1,R,NaN,92.0,88.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
124,LEUVEN_1,50687,2,0,22.0,1,R,NaN,124.0,121.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
125,LEUVEN_1,50688,2,0,21.0,1,R,NaN,98.0,103.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN


In [97]:
df_td_lt18_m = df.loc[(df['SEX'] == 1) & (df['AGE_AT_SCAN'] <=18) & (df['DSM_IV_TR'] == 0) ]

In [98]:
df_td_lt18_m.shape

(296, 74)

In [99]:
# table_males_np = table_males.as_matrix(columns=['SUB_ID','DX_GROUP', 'DSM_IV_TR', 'AGE_AT_SCAN' ,'SEX' ,'EYE_STATUS_AT_SCAN'])

In [113]:
# df_td_lt18_m_eyesclosed

df_aut_subid = df_aut_lt18_m_eyesopen.as_matrix(columns=['SUB_ID'])
df_td_subid = df_td_lt18_m_eyesopen.as_matrix(columns=['SUB_ID'])


In [114]:
# df_aut_subid#, df_td_subid

In [115]:
# Now construct a function that takes a list of SUB_ID's and returns the FC Maps
def get_subject_fc_file(subject_id_list,fc_file_path, bugs):
    import re
    
    return_fc_maps = []
    fc_file_list = np.load(fc_file_path)
    for subject_id in subject_id_list:
#         print("For subject: ",subject_id)
        found =  False
        for brain in fc_file_list:
            sub_id_extracted = re.search('.+_subject_id_(\d+)', brain).group(1)
            if str(subject_id) in bugs:
#                 print("In Bugs with subject id ",subject_id)
                found = True
            elif (subject_id == int(sub_id_extracted)):
                found = True
                return_fc_maps.append(brain)
#                 print("Found for subject: ",subject_id)
        if found == False: # Some subject was not found Problem!
            print ('Unable to locate Subject: ',int(subject_id),'extracted: ',int(sub_id_extracted))
            return 0
    return return_fc_maps
    
    
    

In [116]:
# Again:

fc_datasink_name = 'fc_datasink'
import itertools

# itr = (list(itertools.product([0, 1], repeat=3)))

itr = [(1,1,1)]
number_of_subjects = -1
for motion_param_regression, global_signal_regression, band_pass_filtering in itr:
    combination = 'pearcoff_motionRegress' + str(int(motion_param_regression)) + 'filt' + str(int(band_pass_filtering)) + 'global' + str(int(global_signal_regression))
    print("Combination: ",combination)
    print(motion_param_regression, global_signal_regression, band_pass_filtering)
    fc_file_list = opj(base_directory,fc_datasink_name,combination,'fc_map_brain_file_list.npy')
#     print(fc_file_list)
#     apply_fisher = True


    autistic_list = (get_subject_fc_file(df_aut_subid.squeeze(), fc_file_list, bugs))
    print("Number of autistic participants ", len(autistic_list))
    
    td_list = (get_subject_fc_file(df_td_subid.squeeze(), fc_file_list, bugs))
    print("Number of TD participants ", len(td_list))

Combination:  pearcoff_motionRegress1filt1global1
1 1 1
Number of autistic participants  167
Number of TD participants  239


In [104]:
participants_considered = min(len(autistic_list), len(td_list))

# participants_considered = 4

autistic_list = autistic_list[0:participants_considered]
td_list = td_list[0:participants_considered]

In [117]:
autistic_list

['/home1/varunk/results_again_again/fc_motionRegress1filt1global1/_subject_id_0050183/func2std_xform/0050183_fc_map_flirt.nii.gz',
 '/home1/varunk/results_again_again/fc_motionRegress1filt1global1/_subject_id_0050192/func2std_xform/0050192_fc_map_flirt.nii.gz',
 '/home1/varunk/results_again_again/fc_motionRegress1filt1global1/_subject_id_0050272/func2std_xform/0050272_fc_map_flirt.nii.gz',
 '/home1/varunk/results_again_again/fc_motionRegress1filt1global1/_subject_id_0050273/func2std_xform/0050273_fc_map_flirt.nii.gz',
 '/home1/varunk/results_again_again/fc_motionRegress1filt1global1/_subject_id_0050274/func2std_xform/0050274_fc_map_flirt.nii.gz',
 '/home1/varunk/results_again_again/fc_motionRegress1filt1global1/_subject_id_0050275/func2std_xform/0050275_fc_map_flirt.nii.gz',
 '/home1/varunk/results_again_again/fc_motionRegress1filt1global1/_subject_id_0050277/func2std_xform/0050277_fc_map_flirt.nii.gz',
 '/home1/varunk/results_again_again/fc_motionRegress1filt1global1/_subject_id_00502

In [105]:
len(autistic_list),len(td_list)

(40, 40)

In [106]:
# # To Stop execution Raise error:
# raise Exception('Execution stops here!')

In [107]:
# number_of_fcmaps = len(fc_file_list) #184

In [108]:
# number_of_fcmaps

In [109]:
# Author Deepak Singla: singlakdeepak5@gmail.com

import nibabel as nib
import numpy as np
import os
from scipy import stats
from numpy import ma
import scipy.special as special
from statsmodels.stats import multitest

def div0( a, b ):
    '''
    It is meant for ignoring the places where standard deviation 
    is zero.
    '''
    """ ignore / 0, div0( [-1, 0, 1], 0 ) -> [0, 0, 0] """
    with np.errstate(divide='ignore', invalid='ignore'):
        c = np.divide( a, b )
        # c[ ~ np.isfinite( c )] = 0  # -inf inf NaN
    return c

def calc_mean_and_std(ROICorrMaps, n_subjects, ROIAtlasmask, ddof =1, applyFisher = False):
    '''
	Function for calculating the mean and standard 
	deviation of the data. At a time, only one of the nii
	file is loaded and the elements keep on adding as we 
	enumerate over the subjects.
	'''
    mask = nib.load(ROIAtlasmask).get_data()
    mask = ma.masked_object(mask,0).mask
    if (n_subjects != 0):
        f = nib.load(ROICorrMaps[0])
        dimensions = f.get_header().get_data_shape()
        print(dimensions)
    else:
        exit
    mask = np.repeat(mask[:, :, :, np.newaxis], dimensions[3], axis=3)
#     print(ROICorrMaps)

    Sample_mean_Array = np.zeros(dimensions)
    Sample_std_Array = np.zeros(dimensions)
    Sample_mean_Array = ma.masked_array(Sample_mean_Array, 
                                       mask = mask, 
                                        fill_value = 0)
    Sample_std_Array = ma.masked_array(Sample_std_Array, 
                                      mask = mask , 
                                       fill_value = 0)
    for count, subject in enumerate(ROICorrMaps):
        Corr_data = nib.load(subject).get_data()
#         dimensions = Corr_data.get_header().get_data_shape()
#         mask = np.repeat(mask[:, :, :, np.newaxis], dimensions[3], axis=3)
        Corr_data = ma.masked_array(Corr_data, mask = mask)
        if applyFisher:
            Corr_data = np.arctanh(Corr_data)
        
        Sample_mean_Array += Corr_data
        Sample_std_Array += np.square(Corr_data)
        print('Done subject ', count+1)                                                                                                                                                                                                       
    Sample_mean_Array /= n_subjects
    Sample_std_Array = np.sqrt((Sample_std_Array - n_subjects*np.square(Sample_mean_Array))/(n_subjects - ddof))
    return Sample_mean_Array,Sample_std_Array

def calc_mean_and_std_if_npy(ROICorrMaps, n_subjects, ddof =1, applyFisher = False):
    '''
    Function to be used if the file is given in the format 
    No of ROIs versus All brain voxels in the ROI mapped.
    '''
    print(ROICorrMaps)
    initialize = np.load(ROICorrMaps[0])
    initialize = ma.masked_array(initialize)
    if applyFisher:
        initialize = np.arctanh(initialize)
    Sample_mean_Array = ma.masked_array(initialize, 
                                        fill_value = 0)
    Sample_std_Array = ma.masked_array(np.square(initialize), 
                                       fill_value = 0)
    del initialize
    print('Done subject ', 0)
    for count, subject in enumerate(ROICorrMaps[1:]):

        Corr_data = np.load(subject)
        Corr_data = ma.masked_array(Corr_data) 
        if applyFisher:
            Corr_data = np.arctanh(Corr_data)
        Sample_mean_Array += Corr_data
        Sample_std_Array += np.square(Corr_data)
        print('Done subject ', count+1)                                                                                                                                                                                               
    Sample_mean_Array /= n_subjects
    Sample_std_Array = np.sqrt((Sample_std_Array - n_subjects*np.square(Sample_mean_Array))/(n_subjects - ddof))
    return Sample_mean_Array,Sample_std_Array

def _ttest_1samp(Sample_mean_Array, Sample_std_Array, n_subjects, PopMean = 0.0):
    ttest_1samp_for_all = div0((Sample_mean_Array - PopMean) \
                            * np.sqrt(n_subjects), Sample_std_Array)
    df = n_subjects - 1
    # pval = stats.t.sf(np.abs(ttest_1samp_for_all), df)*2
    pval = special.betainc(0.5*df, 0.5, df/ \
            (df + ttest_1samp_for_all*ttest_1samp_for_all)).reshape(ttest_1samp_for_all.shape)
    # ttest_1samp_for_all, pval = ma.filled(ttest_1samp_for_all), ma.filled(pval)

    return ttest_1samp_for_all, pval


def ttest_1samp_for_all_ROIs(ROICorrMaps, 
                                ROIAtlasmask, 
                                PopMean = 0.0, 
                                applyFisher = False):
    '''
    This is the 1 sample t-test for ROI correlation maps.
    df = no of subjects - 1
    * ROICorrMaps is the list of filepaths of ROI correlation 
    maps for a group.
    * Each ROI correlation map has the 4th dimension equal to 
    the number of ROIs.
    * It calculates both the ttest as well as the p values.
    
    QUESTIONS???????????????????????????????????????????????
    For application of the Fisher transform, I saw that it is 
    same as the inverse hyperbolic tangent function. 
    Doubt is regarding the standard deviation of the distribution after
    applying Fisher. It was written that the sd is now 1/sqrt(no_of_subjs - 3).
    So, that means for each voxel or variable, the sd now becomes this.

    Ref: https://docs.scipy.org/doc/numpy/reference/generated/numpy.arctanh.html
     https://en.wikipedia.org/wiki/Fisher_transformation
    TO BE ASKED!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    
    # The ttest will return t value Inf or NaN where the denom is
    # zero. See what to return in these places. Ask tomorrow.
    '''


    n_subjects = len(ROICorrMaps)
    assert (n_subjects>0)
    Sample_mean_Array, Sample_std_Array = calc_mean_and_std(ROICorrMaps, 
                                                            n_subjects,
                                                            ROIAtlasmask, ddof =1,
                                                            applyFisher = applyFisher)
    ttest_1samp_for_all, pval = _ttest_1samp(Sample_mean_Array, 
                                             Sample_std_Array,
                                             n_subjects,
                                             PopMean = PopMean)
    return ttest_1samp_for_all, pval


def ttest_1samp_ROIs_if_npy(ROICorrMaps,
                            PopMean = 0.0,
                            applyFisher = False):
    n_subjects = len(ROICorrMaps)
    assert (n_subjects>0)
    Sample_mean_Array, Sample_std_Array = \
                                calc_mean_and_std_if_npy( ROICorrMaps,
                                                        n_subjects, ddof =1,
                                                        applyFisher = applyFisher)
    return _ttest_1samp(Sample_mean_Array,
                        Sample_std_Array,
                        n_subjects,
                        PopMean = PopMean)


def _ttest_ind(Sample_mean_ArrayA, Sample_var_ArrayA, n_subjectsA,
                Sample_mean_ArrayB,Sample_var_ArrayB, n_subjectsB,
                equal_var = True):
    if equal_var:
        # force df to be an array for masked division not to throw a warning
        df = ma.asanyarray(n_subjectsA + n_subjectsB - 2.0)
        svar = ((n_subjectsA-1)*Sample_var_ArrayA+(n_subjectsB-1)*Sample_var_ArrayB)/ df
        denom = ma.sqrt(svar*(1.0/n_subjectsA + 1.0/n_subjectsB))  # n-D computation here!
    else:
        vn1 = Sample_var_ArrayA/n_subjectsA
        vn2 = Sample_var_ArrayB/n_subjectsB
        df = (vn1 + vn2)**2 / (vn1**2 / (n_subjectsA - 1) + vn2**2 / (n_subjectsB - 1))

        # If df is undefined, variances are zero.
        # It doesn't matter what df is as long as it is not NaN.
        df = np.where(np.isnan(df), 1, df)
        denom = ma.sqrt(vn1 + vn2)

    with np.errstate(divide='ignore', invalid='ignore'):
        ttest_ind = (Sample_mean_ArrayA - Sample_mean_ArrayB) / denom
    pvalues = special.betainc(0.5*df, 0.5, df/(df + ttest_ind*ttest_ind)).reshape(ttest_ind.shape)
    
    # ttest_ind, pvalues = ma.filled(ttest_ind), ma.filled(pvalues)
    return ttest_ind, pvalues,Sample_mean_ArrayA ,Sample_mean_ArrayB

def ttest_ind_samples(ROICorrMapsA, ROICorrMapsB, ROIAtlasmask, 
                    equal_var = True, applyFisher = False):
    '''
    Modified from https://docs.scipy.org/doc/scipy-0.19.1/reference/generated/scipy.stats.ttest_ind.html ,
    https://github.com/scipy/scipy/blob/v0.19.1/scipy/stats/stats.py#L3950-L4072

    Since it didn't support if the data is large and everything can't be loaded at once. So,
    such modification has been made.
    '''

    n_subjectsA = len(ROICorrMapsA)
    n_subjectsB = len(ROICorrMapsB)
    assert (n_subjectsA > 0)
    assert (n_subjectsB > 0)
    Sample_mean_ArrayA, Sample_std_ArrayA = calc_mean_and_std(ROICorrMapsA, 
                                                              n_subjectsA,
                                                              ROIAtlasmask, ddof =1,
                                                              applyFisher = applyFisher)
    Sample_var_ArrayA = np.square(Sample_std_ArrayA)
    del(Sample_std_ArrayA)

    # n_subjectsB = len(ROICorrMapsB)
    Sample_mean_ArrayB, Sample_std_ArrayB = calc_mean_and_std(ROICorrMapsB, 
                                                              n_subjectsB,
                                                              ROIAtlasmask, ddof =1,
                                                              applyFisher = applyFisher)
    Sample_var_ArrayB = np.square(Sample_std_ArrayB)
    del(Sample_std_ArrayB)

    # pvalues = stats.t.sf(np.abs(ttest_ind), df)*2
    return _ttest_ind(Sample_mean_ArrayA, Sample_var_ArrayA, n_subjectsA,
                Sample_mean_ArrayB, Sample_var_ArrayB, n_subjectsB,
                equal_var = equal_var)

def ttest_ind_samples_if_npy(ROICorrMapsA, ROICorrMapsB, equal_var = True, applyFisher = False):
    '''
    Modified from https://docs.scipy.org/doc/scipy-0.19.1/reference/generated/scipy.stats.ttest_ind.html ,
    https://github.com/scipy/scipy/blob/v0.19.1/scipy/stats/stats.py#L3950-L4072

    Since it didn't support if the data is large and everything can't be loaded at once. So,
    such modification has been made.
    '''

    n_subjectsA = len(ROICorrMapsA)
    n_subjectsB = len(ROICorrMapsB)
    assert (n_subjectsA > 0)
    assert (n_subjectsB > 0)
    Sample_mean_ArrayA, Sample_std_ArrayA = calc_mean_and_std_if_npy(ROICorrMapsA, 
                                                              n_subjectsA, ddof = 1,
                                                              applyFisher = applyFisher)
    Sample_var_ArrayA = np.square(Sample_std_ArrayA)
    del(Sample_std_ArrayA)
    Sample_mean_ArrayB, Sample_std_ArrayB = calc_mean_and_std_if_npy(ROICorrMapsB, 
                                                              n_subjectsB, ddof =1,
                                                              applyFisher = applyFisher)
    Sample_var_ArrayB = np.square(Sample_std_ArrayB)
    del(Sample_std_ArrayB)
    # pvalues = stats.t.sf(np.abs(ttest_ind), df)*2
    return _ttest_ind(Sample_mean_ArrayA, Sample_var_ArrayA, n_subjectsA,
                Sample_mean_ArrayB, Sample_var_ArrayB, n_subjectsB,
                equal_var = equal_var)

def convert_ma_to_np(MaskedArrayObj):
    return ma.filled(MaskedArrayObj)

def fdr_correction(pvals):
#     mask = np.ma.masked_object(mask,0).mask # Converts 1 -> 0 & 0 -> 1 in the mask
    brain_pvals = pvals[~pvals.mask]
    
    
    
    


### Create an MNI 3mm brain mask


In [110]:
mask = opj(base_directory,parent_wf_directory,motion_correction_bet_directory,coreg_reg_directory,'resample_mni/MNI152_T1_2mm_brain_resample_mask.nii.gz')

In [111]:
mask

'/home1/varunk/results_again_again/ABIDE1_Preprocess/motion_correction_bet/coreg_reg/resample_mni/MNI152_T1_2mm_brain_resample_mask.nii.gz'

In [112]:
%%time
#  Author Deepak Singla : singlakdeepak5@gmail.com

import os
import numpy as np
import timeit
# start = timeit.default_timer()

# list1 = np.load('Grp1.npy')
# list2 = np.load('Grp2.npy')

apply_fisher = True
list1 = autistic_list[3:8]
list2 = td_list[3:8]
#Tvals, Pvals = ttest.ttest_1samp_for_all_ROIs(list1,'MNI152_T1_2mm_brain_mask.nii.gz')

Tvals, Pvals, meanC1, meanC2 = ttest_ind_samples(list1,list2,mask,applyFisher=apply_fisher)

fdr_mask_05 , fdr_p_vals = fdr_correction(Pvals, mask)

Tvals, Pvals, meanC1, meanC2 = convert_ma_to_np(Tvals), convert_ma_to_np(Pvals), convert_ma_to_np(meanC1), convert_ma_to_np(meanC2)
# stop = timeit.default_timer()
# print( Tvals)

# print (Pvals)



save_destination = hypothesis_test_dir

if not os.path.exists(save_destination):
    os.makedirs(save_destination) # Use makedirs if you want to create a nested directory structure
Tvals_path = opj(save_destination,'Tvals')
Pvals_path = opj(save_destination,'Pvals')
mean1_path = opj(save_destination,'meanC1')
mean2_path = opj(save_destination,'meanC2')

np.save(Tvals_path,Tvals)
np.save(Pvals_path,Pvals)
np.save(mean1_path,meanC1)
np.save(mean2_path,meanC2)


# print ('Time taken = %s'%(stop - start))


/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: DeprecationWarning: get_header method is deprecated.
Please use the ``img.header`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0


(61, 73, 61, 274)
Done subject  1
Done subject  2
Done subject  3
Done subject  4
Done subject  5
(61, 73, 61, 274)
Done subject  1
Done subject  2
Done subject  3


MaskError: Mask and data not compatible: data size is 74155809, mask size is 74427442.

In [ ]:
p = np.load('../results_again_again/hypothesis_test/hypothesis_test_pearcoff_motionRegress0filt0global0/Pvals.npy')

In [ ]:
orig_shape = p.shape
one_vector = p.reshape()

In [ ]:
p.min(),p.max() # Pmax is too high , maybe it is coz of the mask that I made. Coz all the p's almost unique
# Only the max p has highest frequency, it maybe coz they represent the number of non zero voxels 

In [ ]:
uni, counts = np.unique(p, return_counts=True)

In [ ]:
X = zip(uni,counts)

In [ ]:
uni[-1]

In [ ]:
t = np.load('../results_again_again/hypothesis_test/Tvals.npy')

In [ ]:
t.min(), t.max()

In [ ]:
C1 = np.load('../results_again_again/hypothesis_test/pearcoff_motionRegress1filt1global1/meanC1.npy')

In [ ]:
C1.min(),C1.max()

In [ ]:
C2 = np.load('../results_again_again/hypothesis_test/meanC2.npy')

In [ ]:
C2.min(),C2.max()

In [ ]:
p = np.load('../results_again_again/hypothesis_test/pearcoff_motionRegress1filt0global0/Pvals.npy')

In [ ]:
p.min(),p.max()

## Bugs
* LEUVEN_1 	50703 - 0050703_fc_map_flirt.nii.gz - It is the FC map transformed to MNI 3mm Space. But the transformation looks really bad.
* 0050706_fc_map_flirt.nii.gz
* 0050753_fc_map_flirt.nii.gz Coz the original brain was translated to left
* check 0050752 ???

In [ ]:
import os
save_destination = '/home1/varunk/results_again_again/hypothesis_test/hypothesis_test_pearcoff_motionRegress0filt0global0'
os.path.exists(save_destination)

In [ ]:
os.mkdir(save_destination)

In [ ]:
import numpy as np
l = '/home1/varunk/results_again_again/hypothesis_test/hypothesis_test_pearcoff_motionRegress0filt0global0/Pvals.npy'

X = np.load(l)
X.shape